In [11]:
import numpy as np
import matplotlib.pyplot as plt
import functools

In [12]:
import itertools

onos_nodes = ['c1', 'c2']
switch_nodes = [1, 2, 3]
raw_connections_switch_to_controller = [[switch, onos] for switch in switch_nodes for onos in onos_nodes if switch != onos]

connections_switch_to_controller = []

for combination in itertools.combinations(raw_connections_switch_to_controller, len(switch_nodes)):
    if len(set([item[0] for item in combination])) == len(switch_nodes):
        connections_switch_to_controller.append(combination)

print(connections_switch_to_controller)
print(len(connections_switch_to_controller))

[([1, 'c1'], [2, 'c1'], [3, 'c1']), ([1, 'c1'], [2, 'c1'], [3, 'c2']), ([1, 'c1'], [2, 'c2'], [3, 'c1']), ([1, 'c1'], [2, 'c2'], [3, 'c2']), ([1, 'c2'], [2, 'c1'], [3, 'c1']), ([1, 'c2'], [2, 'c1'], [3, 'c2']), ([1, 'c2'], [2, 'c2'], [3, 'c1']), ([1, 'c2'], [2, 'c2'], [3, 'c2'])]
8


In [13]:
import random

switch_nodes = [1, 2, 3]
raw_connections_switch_to_switch = [[node1, node2] for node1 in switch_nodes for node2 in switch_nodes if node1 != node2]
print(raw_connections_switch_to_switch)

delay_switch_to_switch = [[item[0], item[1], random.randint(1, 100)] for item in raw_connections_switch_to_switch]
print(delay_switch_to_switch)

[[1, 2], [1, 3], [2, 1], [2, 3], [3, 1], [3, 2]]
[[1, 2, 18], [1, 3, 34], [2, 1, 45], [2, 3, 33], [3, 1, 71], [3, 2, 71]]


In [14]:
import heapq

def dijkstra(graph, start, end):
    # 初始化距离字典和优先队列
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    priority_queue = [(0, start)]

    # 初始化路径字典
    previous_nodes = {node: None for node in graph}

    while priority_queue:
        # 从优先队列中获取当前距离最小的节点
        current_distance, current_node = heapq.heappop(priority_queue)

        # 如果当前距离大于已知最短距离，则跳过该节点
        if current_distance > distances[current_node]:
            continue

        # 如果到达目标节点，则构建并返回最短路径
        if current_node == end:
            path = []
            while current_node is not None:
                path.append(current_node)
                current_node = previous_nodes[current_node]
            path.reverse()
            return path

        # 遍历当前节点的邻居节点
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight

            # 如果通过当前节点到达邻居节点的距离更短，则更新最短距离和路径
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous_nodes[neighbor] = current_node
                heapq.heappush(priority_queue, (distance, neighbor))

    # 如果无法到达目标节点，则返回空路径
    return None

# 示例使用
# graph = [(1,3),(3,2)]
# graph = {
#     '1': {'3': 54, '4': 10},
#     '2': {'3': 68, '4': 30},
#     '3': {'1': 54, '2': 68},
#     '4': {'1': 10, '2': 30}
# }
graph = {
    '1': {'3': 54},
    '2': {'3': 68},
    '3': {'1': 54, '2': 68}
}

start_node = '1'
end_node = '2'

shortest_path = dijkstra(graph, start_node, end_node)
if shortest_path is not None:
    print(f"Shortest path from {start_node} to {end_node}: {shortest_path}")
else:
    print(f"No path found from {start_node} to {end_node}")

# print(delay_switch_to_switch)
# [[1, 2, 82], [1, 3, 54], [2, 1, 82], [2, 3, 68], [3, 1, 96], [3, 2, 48]]

Shortest path from 1 to 2: ['1', '3', '2']


In [74]:
import random

# onos_nodes = ['c1', 'c2']
# raw_connections_switch_to_switch = [[node1, node2] for node1 in switch_nodes for node2 in switch_nodes if node1 != node2]

numbers = [number for number in switch_nodes]
# print(numbers)

placement = [(node, number) for node, number in zip(onos_nodes, random.sample(numbers, len(onos_nodes)))]
print(placement)

[('c1', 3), ('c2', 2)]


In [301]:
import random

graph = {
    '1': {'3': 54},
    '2': {'3': 68},
    '3': {'1': 54, '2': 68}
}

convert_graph_to_array = [[int(source), int(target), weight] if target != '0' else [int(source), int(target), 0] for source, edges in graph.items() for target, weight in edges.items()]
# print(convert_graph_to_array)
# [[1, 3, 54], [2, 3, 68], [3, 1, 54], [3, 2, 68]]
delay = []
assign_delay = 0

for onos in placement:
    for switch in switch_nodes:
        if onos[1] == switch:
            delay.append([onos[0], switch, 0])
        else:
            for item in convert_graph_to_array:
                if item[0] == switch and item[1] == onos[1]:
                        delay.append([onos[0], switch, item[2]])
                if item[0] == switch and item[1] != onos[1]:
                    path = dijkstra(graph, str(onos[1]), str(item[0]))
                    if len(path) > 2:
                        for node_along_path in range(len(path)):
                            slice_start = node_along_path
                            slice_end = node_along_path + 2
                            if slice_end > len(path):
                                break
                            else:
                                pick_delay = [item[2] for item in convert_graph_to_array if [str(item[0]), str(item[1])] == path[slice_start:slice_end]]
                                assign_delay += int(pick_delay[0])
                        delay.append([onos[0], switch, assign_delay])
print(delay)
# delay = [('c1', 1, 54),('c1', 2, 68),('c1', 3, 0),('c2', 1, 122),('c2', 2, 0),('c2', 3, 68)]


# switch_nodes = [1, 2, 3]
# raw_connections_switch_to_switch = [[node1, node2] for node1 in switch_nodes for node2 in switch_nodes if node1 != node2]
# [[1, 2], [1, 3], [2, 1], [2, 3], [3, 1], [3, 2]]

# delay_switch_to_switch = [[item[0], item[1], random.randint(1, 100)] for item in raw_connections_switch_to_switch]
# print(delay_switch_to_switch)
# [[1, 2, 18], [1, 3, 34], [2, 1, 45], [2, 3, 33], [3, 1, 71], [3, 2, 71]]

[['c1', 1, 54], ['c1', 2, 68], ['c1', 3, 0], ['c2', 1, 122], ['c2', 2, 0], ['c2', 3, 68]]


In [177]:
path = ['1', '3', '2']
convert_graph_to_array = [[1, 3, 54], [2, 3, 68], [3, 1, 54], [3, 2, 68]]

result = []
value = 0

for item in range(len(path)):
    slice_start = item
    slice_end = item + 2
    if slice_end > len(path):
        break
    else:
        pick_delay = [item[2] for item in convert_graph_to_array if [str(item[0]), str(item[1])] == path[slice_start:slice_end]]
        value += int(pick_delay[0])
        
print(value)

122


In [305]:
# delay = [('c1', 1, 54),('c1', 2, 68),('c1', 3, 0),('c2', 1, 122),('c2', 2, 0),('c2', 3, 68)]
# shortest_path = ['1', '3', '2']

sum = 0
for node_along_shortest_path in range(len(shortest_path)):
    slice_start = node_along_shortest_path
    slice_end = node_along_shortest_path + 2
    if slice_end > len(shortest_path):
        break
    else:
        pick_delay = [item[2] for item in convert_graph_to_array if [str(item[0]), str(item[1])] == shortest_path[slice_start:slice_end]]
        sum += int(pick_delay[0])

print(sum)
fst = 4*sum

print(fst)

122
488
